In [1]:
import pandas as pd
import ast

In [2]:
# Se abre los archivos steam, df_items y reviuws
steam = pd.read_csv('steam.csv')
item = pd.read_csv('df_items.csv')
reviu = pd.read_csv('reviuws.csv')

In [3]:
# Chekeamos como estan compuestos los dataframe
print(steam.shape)
print(item.shape)
print(reviu.shape)

(28728, 6)
(3210544, 4)
(59305, 10)


Observamos la informacion de los dataframes


In [4]:
steam.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28728 entries, 0 to 28727
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Unnamed: 0    28728 non-null  int64 
 1   id            28728 non-null  int64 
 2   genres        28728 non-null  object
 3   app_name      28728 non-null  object
 4   release_date  28728 non-null  object
 5   year          28728 non-null  int64 
dtypes: int64(3), object(3)
memory usage: 1.3+ MB


In [5]:
item.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3210544 entries, 0 to 3210543
Data columns (total 4 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   Unnamed: 0        int64 
 1   item_id           int64 
 2   playtime_forever  int64 
 3   user_id           object
dtypes: int64(3), object(1)
memory usage: 98.0+ MB


In [6]:
reviu.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59305 entries, 0 to 59304
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Unnamed: 0         59305 non-null  int64 
 1   funny              8151 non-null   object
 2   last_edited        6140 non-null   object
 3   item_id            59305 non-null  int64 
 4   helpful            59305 non-null  object
 5   recommend          59305 non-null  bool  
 6   user_id            59305 non-null  object
 7   user_url           59305 non-null  object
 8   sentiment_analisy  59305 non-null  int64 
 9   date               59305 non-null  object
dtypes: bool(1), int64(3), object(6)
memory usage: 4.1+ MB


Se crean dataframes con campos requeridos para el desarrollo da las consultas 

In [7]:
# Se crea un dataframe genero, con las campos 'id','genres','year'
genero = steam[['id','genres','year']].copy()

In [8]:
# Se crea un dataframe play con los campos, 'item_id','user_id','playtime_forever'
play = item[['item_id','user_id','playtime_forever']].copy()

In [9]:
# Se realiza la union de los dataframe anteriores para crear 'generos', el cual se utiliza para una consulta
generos = pd.merge(genero, play, left_on='id', right_on='item_id')


In [10]:
# Del campo genres utilizando ast.literal_eval para las listas de generos
generos['genres'] = generos['genres'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)



In [11]:
# Se crea cada genero como un nuevo registro
generos = generos.explode('genres')

In [12]:
generos.head()

,id,genres,year,item_id,user_id,playtime_forever
0,282010,Action,1997,282010,r3ap3r78,0
0,282010,Indie,1997,282010,r3ap3r78,0
0,282010,Racing,1997,282010,r3ap3r78,0
1,282010,Action,1997,282010,Telayne,47
1,282010,Indie,1997,282010,Telayne,47


In [64]:
# Exportamos el archivo csv
generos.to_csv('generos.csv')

In [13]:
generos1 = generos[['user_id','genres','year','playtime_forever']].copy()

In [14]:
# Se toma la decision de trabajar con el 40% de los datos, para disminuir su tamaño de memoria 
# en aras de realizar el ejercicio practico y permite subir el archivo
sample_size = int(0.40 * len(generos1)) 
generos1 = generos1.sample(n=sample_size, random_state=42) 

In [70]:
generos1.to_csv('generos1.csv')

In [15]:

# Se crea la funcion
def PlayTimeGenre( genero : str ):

    '''Debe devolver año con mas horas jugadas para dicho género.'''
    
    genero_df = generos1[generos1['genres'].str.contains(genero, case=False)]
    
    anio_con_max_horas = genero_df.groupby('year')['playtime_forever'].sum().idxmax()
    
    return {f'Año con mas horas jugadas del genero {genero}':anio_con_max_horas}



In [16]:
# Se prueba la funcion 
PlayTimeGenre('Action')

{'Año con mas horas jugadas del genero Action': 2012}

In [17]:
# Se crea la funcion
def UserForGenre( genero : str ):
    ''' Debe devolver el usuario que acumula más horas jugadas para el género 
    dado y una lista de la acumulación de horas jugadas por año.'''
    
    genero_df = generos1[generos1['genres'].str.contains(genero, case=False)]
    
    usuario_mas_horas = genero_df.groupby('user_id')['playtime_forever'].sum().idxmax()
    
    usuario_genero_df = genero_df[genero_df['user_id'] == usuario_mas_horas]
    
    acumulacion_horas_por_anio = usuario_genero_df.groupby('year')['playtime_forever'].sum().reset_index()

    lista_acumulacion_horas = [{'año': año, 'horas': horas} for año, horas in zip(acumulacion_horas_por_anio['year'], acumulacion_horas_por_anio['playtime_forever'])]

    
    return {f'Usuario con mas horas jugadas para el genero {genero}': usuario_mas_horas, 'Horas jugadas': lista_acumulacion_horas}



In [18]:
# Se prueba la funcion 
UserForGenre('Action')

{'Usuario con mas horas jugadas para el genero Action': 'stopgovtcorruption',
 'Horas jugadas': [{'año': 2006, 'horas': 234508},
  {'año': 2007, 'horas': 0},
  {'año': 2008, 'horas': 0},
  {'año': 2009, 'horas': 342805},
  {'año': 2010, 'horas': 0},
  {'año': 2011, 'horas': 1708},
  {'año': 2013, 'horas': 1177},
  {'año': 2014, 'horas': 27440},
  {'año': 2015, 'horas': 3671}]}

In [19]:
# Se Crea un datagrames año_game, compuesto con los campos 'id', 'app_name','year'
# Se renombra la columna 'id' a item_id
# y se eliminan los duplicados

año_game = steam[['id','app_name', 'year']].copy()
año_game = año_game.rename(columns={'id':'item_id'})
año_game = año_game.drop_duplicates()
año_game.head()

,item_id,app_name,year
0,761140,Lost Summoner Kitty,2018
1,643980,Ironbound,2018
2,670290,Real Pool 3D - Poolians,2017
3,767400,弹炸人2222,2017
4,772540,Battle Royale Trainer,2018


In [20]:
# Se crea el dataframe recomendado, con los campos 'item_id','recommend','sentiment_analisy'
recomendado = reviu[['item_id','recommend','sentiment_analisy']].copy()
recomendado.head()

,item_id,recommend,sentiment_analisy
0,1250,True,2
1,22200,True,2
2,43110,True,2
3,251610,True,2
4,227300,True,0


In [21]:
# Se une el Dataframe año_game a recomendado para el desarrollo de las sgtes funcion
recomendado = recomendado.merge(año_game, on='item_id')
recomendado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50387 entries, 0 to 50386
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   item_id            50387 non-null  int64 
 1   recommend          50387 non-null  bool  
 2   sentiment_analisy  50387 non-null  int64 
 3   app_name           50387 non-null  object
 4   year               50387 non-null  int64 
dtypes: bool(1), int64(3), object(1)
memory usage: 1.6+ MB


In [23]:
recomendado.to_csv('recomendado.csv')

In [22]:

def UsersRecommend(año:int):
    '''Devuelve el top 3 de juegos MÁS recomendados por usuarios para el año 
    dado. (reviews.recommend = True y comentarios positivos/neutrales) '''

    filtered_df = recomendado[(recomendado['year'] == año) & (recomendado['recommend'] == True) & (recomendado['sentiment_analisy'] >= 1)]
    
    # Agrupar y contar los juegos recomendados
    game_counts = filtered_df['app_name'].value_counts().reset_index()
    game_counts.columns = ['app_name', 'count']
    
    # Ordenar por la cantidad de recomendaciones en orden descendente
    sorted_games = game_counts.sort_values(by='count', ascending=False)
    
    # Tomar los 3 juegos principales
    top_3_games = sorted_games.head(3)
    
    # Crear la lista de diccionarios en el formato deseado
    result = [{"Puesto {}: {}".format(i+1, game['app_name'])} for i, game in top_3_games.iterrows()]
    
    return result


In [23]:
UsersRecommend(2014)

[{'Puesto 1: Loadout'}, {'Puesto 2: Goat Simulator'}, {'Puesto 3: Insurgency'}]

In [24]:


def UsersNotRecommend(año:int):

    ''' Devuelve el top 3 de juegos MENOS recomendados por usuarios para el año dado. 
    (reviews.recommend = False y comentarios negativos)'''


    filtered_df = recomendado[(recomendado['year'] == año) & (recomendado['recommend'] == False) & (recomendado['sentiment_analisy'] == 0)]
    
    # Agrupar y contar los juegos menos recomendados
    game_counts = filtered_df['app_name'].value_counts().reset_index()
    game_counts.columns = ['app_name', 'count']
    
    # Ordenar por la cantidad de juegos menos recomendados en orden descendente
    sorted_games = game_counts.sort_values(by='count', ascending=False)
    
    # Tomar los 3 juegos principales
    top_3_least_recommended = sorted_games.head(3)
    
    # Crear la lista de diccionarios en el formato deseado
    result = [{"Puesto {}: {}".format(i+1, game['app_name'])} for i, game in top_3_least_recommended.iterrows()]
    
    return result


In [25]:
UsersNotRecommend(2014)

[{'Puesto 1: Loadout'},
 {'Puesto 2: Call of Duty®: Ghosts'},
 {'Puesto 3: theHunter Classic'}]

In [26]:
# Se crea el datagrame sentimiento, con los campos ['item_id','sentiment_analisy'
sentimento = reviu[['item_id','sentiment_analisy']].copy()
sentimento = sentimento.drop_duplicates()
sentimento.head()

,item_id,sentiment_analisy
0,1250,2
1,22200,2
2,43110,2
3,251610,2
4,227300,0


In [27]:
# Se une el Dataframe año_game a df_reviu
sentimento = sentimento.merge(año_game, on='item_id')
sentimento.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5465 entries, 0 to 5464
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   item_id            5465 non-null   int64 
 1   sentiment_analisy  5465 non-null   int64 
 2   app_name           5465 non-null   object
 3   year               5465 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 170.9+ KB


In [43]:
# Se guarda a archivo csv
sentimento.to_csv('sentiment.csv')

In [28]:
#Se crea la funcion
def sentiment_analysis(año:int):
    ''' Según el año de lanzamiento, se devuelve una lista con la cantidad 
# de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento.'''

    año_reviu = sentimento[sentimento['year'] == año]
    
    conteo_sentiment = {'Negative': 0, 'Neutral': 0, 'Positive': 0}
    
    for index, row in año_reviu.iterrows():
        sentiment = row['sentiment_analisy']
        categoria = ''
        
        if sentiment == 0:
            categoria = 'Negative'
        elif sentiment == 1:
            categoria = 'Neutral'
        elif sentiment == 2:
            categoria = 'Positive'
        
        # Incrementar el contador correspondiente en el diccionario
        conteo_sentiment[categoria] += 1
    
    return conteo_sentiment

In [29]:
# Se prueba la funcion
sentiment_analysis(2014)

{'Negative': 331, 'Neutral': 259, 'Positive': 468}